# Time Series Analisis Audio Multilable

## Import Library

In [1]:

!pip install pandas librosa tsfel tqdm scikit-learn joblib

import os
import pandas as pd
import librosa
import tsfel
from tqdm import tqdm
import warnings

import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import make_scorer, recall_score, f1_score, mean_absolute_percentage_error
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import joblib

# Mengabaikan peringatan saat load file audio (opsional tapi disarankan)
warnings.filterwarnings('ignore', category=UserWarning, module='librosa')

print("Cell 1: Semua library berhasil di-import.")

Cell 1: Semua library berhasil di-import.


## Standarisasi Audio (Convert ke Wav)

In [2]:

!mkdir dataset_wav

print("Memulai konversi audio ke .wav (16000 Hz, mono)...")

!for %f in ("data\buka fikri\*.mp3") do ffmpeg -i "%f" -ac 1 -ar 16000 "dataset_wav\%~nf.wav"
!for %f in ("data\tutup fikri\*.mp3") do ffmpeg -i "%f" -ac 1 -ar 16000 "dataset_wav\%~nf.wav"
!for %f in ("data\buka fauzan\*.aac") do ffmpeg -i "%f" -ac 1 -ar 16000 "dataset_wav\%~nf.wav"
!for %f in ("data\tutup fauzan\*.aac") do ffmpeg -i "%f" -ac 1 -ar 16000 "dataset_wav\%~nf.wav"

print("\nStandardisasi selesai. Folder 'dataset_wav' siap.")

Memulai konversi audio ke .wav (16000 Hz, mono)...



C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-14.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-08-14.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-24.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-08-24.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-30.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-08-30.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-36.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-08-36.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-43.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-08-43.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\buka fikri\voice-recorder-2025-11-02--13-08-50.mp3" -ac

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en


C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-19-40.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-19-40.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-19-44.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-19-44.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-19-48.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-19-48.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-19-51.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-19-51.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-19-55.mp3" -ac 1 -ar 16000 "dataset_wav\voice-recorder-2025-11-02--13-19-55.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fikri\voice-recorder-2025-11-02--13-20-00.mp

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

...

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en

ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --en


C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052203.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052203.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052207.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052207.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052211.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052211.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052215.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052215.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052219.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052219.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052242.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052242.wav" 

C:\Users\FIKRI\Documents\REPO\PSD\psd>ffmpeg -i "data\tutup fauzan\20251031_052247.aac" -ac 1 -ar 16000 "dataset_wav\20251031_052247.wav" 

C:\Users\FIKRI\Docu


Standardisasi selesai. Folder 'dataset_wav' siap.



  Stream #0:0 -> #0:0 (aac (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to 'dataset_wav\tutup70.wav':
  Metadata:
    ISFT            : Lavf62.3.100
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s
    Metadata:
      encoder         : Lavc62.11.100 pcm_s16le
[out#0/wav @ 0000021a49307200] video:0KiB audio:42KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.180991%
size=      42KiB time=00:00:01.34 bitrate= 256.5kbits/s speed= 141x elapsed=0:00:00.00    
ffmpeg version 8.0-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt 

## Membuat Metadata Multilable

In [3]:


import os
import pandas as pd

ORIGINAL_DATA_PATH = "data/" # Path ke folder 'data' Anda
metadata_list = []

# Definisikan pemetaan dari nama folder ke label multi-hot
folder_mappings = {
    # Pastikan nama string ini SAMA PERSIS dengan nama folder Anda
    "buka fikri":  {'buka': 1, 'tutup': 0, 'fikri': 1, 'fauzan': 0},
    "tutup fikri": {'buka': 0, 'tutup': 1, 'fikri': 1, 'fauzan': 0},
    "buka fauzan": {'buka': 1, 'tutup': 0, 'fikri': 0, 'fauzan': 1},
    "tutup fauzan":{'buka': 0, 'tutup': 1, 'fikri': 0, 'fauzan': 1}
}

print(f"Membaca struktur folder asli dari {ORIGINAL_DATA_PATH}...")

# Loop melalui setiap folder yang kita definisikan
for folder_name, labels in folder_mappings.items():
    # Menggunakan os.path.join agar aman di Windows
    folder_path = os.path.join(ORIGINAL_DATA_PATH, folder_name)
    
    if not os.path.isdir(folder_path):
        print(f"Warning: Folder '{folder_path}' tidak ditemukan. Dilewati.")
        continue
        
    print(f"Memproses folder: {folder_name}")
    
    # Loop melalui setiap file di dalam folder tersebut
    for original_filename in os.listdir(folder_path):
        # Dapatkan nama file tanpa ekstensi (.mp3 atau .aac)
        base_name, ext = os.path.splitext(original_filename)
        
        # Kita hanya peduli file audio
        if ext.lower() not in ['.mp3', '.aac']:
            continue
            
        new_wav_filename = base_name + ".wav"
        
        # Buat baris metadata baru
        new_row = {
            'file_name': new_wav_filename,
            'buka': labels['buka'],
            'tutup': labels['tutup'],
            'fikri': labels['fikri'],
            'fauzan': labels['fauzan']
        }
        metadata_list.append(new_row)

# --- Simpan ke CSV ---
if not metadata_list:
    print("\nERROR: Tidak ada file audio (.mp3/.aac) yang ditemukan di subfolder 'data/'.")
    print("Pastikan folder 'data' Anda ada di lokasi yang sama dengan notebook ini.")
else:
    df_metadata = pd.DataFrame(metadata_list)
    df_metadata.to_csv("metadata_anda.csv", index=False)
    
    print(f"\nFile 'metadata_anda.csv' berhasil dibuat dengan {len(df_metadata)} baris.")
    print("\nContoh 5 baris pertama:")
    display(df_metadata.head()) 
    print("\nContoh 5 baris terakhir:")
    display(df_metadata.tail())

Membaca struktur folder asli dari data/...
Memproses folder: buka fikri
Memproses folder: tutup fikri
Memproses folder: buka fauzan
Memproses folder: tutup fauzan

File 'metadata_anda.csv' berhasil dibuat dengan 400 baris.

Contoh 5 baris pertama:


,file_name,buka,tutup,fikri,fauzan
0,voice-recorder-2025-11-02--13-08-14.wav,1,0,1,0
1,voice-recorder-2025-11-02--13-08-24.wav,1,0,1,0
2,voice-recorder-2025-11-02--13-08-30.wav,1,0,1,0
3,voice-recorder-2025-11-02--13-08-36.wav,1,0,1,0
4,voice-recorder-2025-11-02--13-08-43.wav,1,0,1,0



Contoh 5 baris terakhir:


,file_name,buka,tutup,fikri,fauzan
395,tutup51.wav,0,1,0,1
396,tutup60.wav,0,1,0,1
397,tutup70.wav,0,1,0,1
398,tutup80.wav,0,1,0,1
399,tutup90.wav,0,1,0,1


## Memuat Data Audio ke Python

In [4]:


METADATA_PATH = "metadata_anda.csv"
AUDIO_FOLDER_PATH = "dataset_wav/"
TARGET_SR = 16000

print(f"Membaca file metadata dari {METADATA_PATH}...")
try:
    df_metadata = pd.read_csv(METADATA_PATH)
except FileNotFoundError:
    print(f"ERROR: File {METADATA_PATH} tidak ditemukan.")
    raise

data_list = []
print(f"Memulai proses pembacaan {len(df_metadata)} file audio dari {AUDIO_FOLDER_PATH}...")

for index, row in tqdm(df_metadata.iterrows(), total=df_metadata.shape[0]):
    file_path = os.path.join(AUDIO_FOLDER_PATH, row['file_name'])
    
    try:
        audio_signal, sampling_rate = librosa.load(file_path, sr=TARGET_SR)
        
        data_list.append({
            'filename': row['file_name'],
            'signal': audio_signal,
            'sampling_rate': sampling_rate,
            'label_buka': row['buka'],
            'label_tutup': row['tutup'],
            'label_fikri': row['fikri'],
            'label_fauzan': row['fauzan']
        })
        
    except Exception as e:
        print(f"\nWarning: Gagal membaca file {file_path}: {e}")
        
print(f"\nProses pembacaan data selesai.")
print(f"Total file berhasil dibaca: {len(data_list)}")

if data_list:
    print("\nContoh data pertama yang dimuat:")
    print(data_list[0])

Membaca file metadata dari metadata_anda.csv...


Memulai proses pembacaan 400 file audio dari dataset_wav/...

  0%|                                                              | 0/400 [00:00<?, ?it/s]

  0%|▏                                                   | 1/400 [00:09<1:01:09,  9.20s/it]

 12%|██████▏                                              | 47/400 [00:09<00:49,  7.14it/s]

...

 66%|██████████████████████████████████▌                 | 266/400 [00:09<00:01, 92.44it/s]

 76%|███████████████████████████████████████            | 306/400 [00:09<00:00, 119.84it/s]

 88%|████████████████████████████████████████████▊      | 351/400 [00:10<00:00, 157.84it/s]

100%|████████████████████████████████████████████████████| 400/400 [00:10<00:00, 39.54it/s]


Proses pembacaan data selesai.
Total file berhasil dibaca: 400

Contoh data pertama yang dimuat:
{'filename': 'voice-recorder-2025-11-02--13-08-14.wav', 'signal': array([ 0.        ,  0.        ,  0.        , ..., -0.00015259,
       -0.00039673, -0.00042725], dtype=float32), 'sampling_rate': 16000, 'label_buka': 1, 'label_tutup': 0, 'label_fikri': 1, 'label_fauzan': 0}


## Ekstraksi Fitur TSFEL

In [5]:


def extract_features_tsfel(data_list):
    cfg = tsfel.get_features_by_domain()
    
    feature_dfs = []
    print(f"\nMemulai proses ekstraksi fitur TSFEL untuk {len(data_list)} file...\n")

    for data in tqdm(data_list, total=len(data_list)):
        signal = data['signal']
        sr = data['sampling_rate']
        
        try:
            features = tsfel.time_series_features_extractor(cfg, signal, fs=sr, verbose=0)
            
            features['filename'] = data['filename']
            features['label_buka'] = data['label_buka']
            features['label_tutup'] = data['label_tutup']
            features['label_fikri'] = data['label_fikri']
            features['label_fauzan'] = data['label_fauzan']
            
            feature_dfs.append(features)
            
        except Exception as e:
            print(f"\nError saat ekstraksi fitur {data['filename']}: {e}")

    if not feature_dfs:
        print("Tidak ada fitur yang berhasil diekstrak.")
        return pd.DataFrame()
        
    feature_df = pd.concat(feature_dfs, ignore_index=True)
    
    print(f"\nEkstraksi fitur selesai!")
    print(f"Total data fitur: {feature_df.shape[0]} baris, {feature_df.shape[1]} kolom")
    
    return feature_df

# --- Bagian Eksekusi ---
if 'data_list' in locals():
    print("Menjalankan ekstraksi fitur pada 'data_list' yang sudah dimuat...")
    features_df = extract_features_tsfel(data_list)
    
    print("\nContoh 5 baris pertama dari DataFrame fitur (features_df):")
    display(features_df.head())
else:
    print("ERROR: Variabel 'data_list' tidak ditemukan. Jalankan ulang Cell 4.")

Menjalankan ekstraksi fitur pada 'data_list' yang sudah dimuat...

Memulai proses ekstraksi fitur TSFEL untuk 400 file...



  0%|                                                              | 0/400 [00:00<?, ?it/s]

  0%|▏                                                     | 1/400 [00:01<06:43,  1.01s/it]

  0%|▎                                                     | 2/400 [00:02<06:46,  1.02s/it]

  1%|▍                                                     | 3/400 [00:03<06:45,  1.02s/it]

...

  5%|██▌                                                  | 19/400 [00:26<08:15,  1.30s/it]

  5%|██▋                                                  | 20/400 [00:28<08:37,  1.36s/it]

  5%|██▊                                                  | 21/400 [00:29<07:30,  1.19s/it]

  5%|██▊                                                  | 21/400 [00:29<09:01,  1.43s/it]

KeyboardInterrupt: 

## Seleksi Fitur Information Gain

In [ ]:


from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def clean_and_select_features(features_df):
    """
    Membersihkan data (NaN, inf) dan melakukan seleksi fitur
    menggunakan Information Gain (Mutual Information).
    """
    
    print("Memulai pembersihan data dan seleksi fitur...")
    
    # 1. Pisahkan fitur (X) dan label (y)
    label_cols = ['label_buka', 'label_tutup', 'label_fikri', 'label_fauzan']
    # Kita hapus 'filename' dan semua kolom label
    X = features_df.drop(columns=['filename'] + label_cols, errors='ignore')
    y = features_df[label_cols]
    
    # Simpan nama fitur untuk nanti
    feature_names = X.columns
    
    # 2. Tangani missing value (NaN) & infinity (inf)
    # TSFEL kadang menghasilkan 'inf', ganti dengan 'NaN' dulu
    X = X.replace([np.inf, -np.inf], np.nan)
    
    if X.isnull().sum().sum() > 0:
        print(f"Ditemukan {X.isnull().sum().sum()} missing values. Mengisi dengan median...")
        imputer = SimpleImputer(strategy='median')
        # fit_transform mengembalikan numpy array, jadi kita simpan nama kolomnya
        X_imputed = imputer.fit_transform(X)
    else:
        print("Tidak ada missing values. Melanjutkan...")
        X_imputed = X.values # Ubah ke numpy array agar konsisten

    # 3. Hitung Information Gain (Mutual Information) untuk SETIAP label
    # Kita TIDAK perlu LabelEncoder karena label kita sudah 0/1
    print("Menghitung Information Gain untuk 4 label...")
    
    # discrete_features='auto' bagus untuk menangani fitur TSFEL
    ig_buka = mutual_info_classif(X_imputed, y['label_buka'], discrete_features='auto')
    ig_tutup = mutual_info_classif(X_imputed, y['label_tutup'], discrete_features='auto')
    ig_fikri = mutual_info_classif(X_imputed, y['label_fikri'], discrete_features='auto')
    ig_fauzan = mutual_info_classif(X_imputed, y['label_fauzan'], discrete_features='auto')
    
    # 4. Gabungkan hasil ke DataFrame
    ig_df = pd.DataFrame({
        'Feature': feature_names,
        'IG_Buka': ig_buka,
        'IG_Tutup': ig_tutup,
        'IG_Fikri': ig_fikri,
        'IG_Fauzan': ig_fauzan
    })
    
    # Buat kolom rata-rata IG untuk mengurutkan fitur terbaik secara keseluruhan
    ig_df['IG_Average'] = ig_df[['IG_Buka', 'IG_Tutup', 'IG_Fikri', 'IG_Fauzan']].mean(axis=1)
    
    # Urutkan dari yang paling informatif
    ig_df = ig_df.sort_values(by='IG_Average', ascending=False).reset_index(drop=True)
    
    # --- 5. VISUALISASI (Sesuai permintaan Anda) ---
    print("Membuat visualisasi untuk 20 fitur terbaik...")
    
    top_n = 20
    top_features = ig_df.head(top_n)
    
    plt.figure(figsize=(12, 10))
    sns.barplot(
        x='IG_Average', 
        y='Feature', 
        data=top_features, 
        palette='viridis'
    )
    plt.title(f'Top {top_n} Fitur Paling Informatif (Rata-rata Information Gain)', fontsize=16)
    plt.xlabel('Rata-rata Skor IG (Semakin tinggi semakin baik)', fontsize=12)
    plt.ylabel('Nama Fitur (dari TSFEL)', fontsize=12)
    plt.tight_layout()
    plt.show() # Tampilkan plot!
    
    return ig_df, X_imputed, y

# --- Bagian Eksekusi ---
if 'features_df' in locals():
    # Jalankan fungsi dan simpan hasilnya ke variabel baru
    ig_df, X_imputed, y_labels = clean_and_select_features(features_df)
    
    print("\nSeleksi fitur selesai.")
    print("Contoh 5 fitur TERBAIK berdasarkan rata-rata IG:")
    display(ig_df.head())
    
    # Simpan nama-nama fitur terbaik untuk digunakan di cell training
    # Ambil 100 fitur terbaik (atau sesuai kebutuhan)
    TOP_N_FEATURES = 100 
    selected_feature_names = ig_df.head(TOP_N_FEATURES)['Feature'].tolist()
    print(f"\nDisimpan {len(selected_feature_names)} nama fitur terbaik untuk training.")
    
else:
    print("ERROR: Variabel 'features_df' tidak ditemukan. Jalankan ulang Cell 5.")

## Pelatihan & Menyimpan Model Multilabel

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
import numpy as np
import joblib

print("Memulai persiapan data untuk pelatihan model...")

# 1. Ambil 100 fitur terbaik yang sudah kita pilih di cell sebelumnya
# Pastikan variabel 'selected_feature_names' ada dari Cell 6
if 'selected_feature_names' not in locals():
    print("ERROR: 'selected_feature_names' tidak ditemukan. Jalankan ulang Cell 6.")
    raise NameError("'selected_feature_names' not defined")

if 'features_df' not in locals():
    print("ERROR: 'features_df' tidak ditemukan. Jalankan ulang Cell 5.")
    raise NameError("'features_df' not defined")

# 2. Buat dataset final X dan y
label_cols = ['label_buka', 'label_tutup', 'label_fikri', 'label_fauzan']
X = features_df[selected_feature_names] # Hanya ambil 100 fitur terbaik
y = features_df[label_cols]

print(f"Dataset final dibuat dengan {X.shape[1]} fitur dan {X.shape[0]} data.")

# 3. Lakukan pembersihan (Imputasi) HANYA pada 100 fitur ini
# Ini penting agar data 'live' nanti bisa diproses dengan cara yang sama
X = X.replace([np.inf, -np.inf], np.nan)
imputer = SimpleImputer(strategy='median')
X_final = imputer.fit_transform(X)
y_final = y.values # Konversi y ke numpy array

# 4. Split data (Train/Test)
# Kita akan gunakan 80% data untuk melatih, 20% untuk menguji
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42
)

print(f"Data dibagi: {len(X_train)} data latih, {len(X_test)} data uji.")

# 5. Definisikan dan Latih Model (Random Forest)
# n_estimators=100 artinya kita membangun 100 "pohon"
# random_state=42 agar hasilnya konsisten setiap kali dijalankan
print("Memulai pelatihan RandomForestClassifier (Multilabel)...")
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model!
model.fit(X_train, y_train)

print("Pelatihan model selesai!")

# 6. Evaluasi Model
print("\n--- HASIL EVALUASI MODEL PADA DATA UJI ---")
y_pred = model.predict(X_test)

# Akurasi (Subset Accuracy):
# Ini adalah skor yang SANGAT KETAT. 
# Model harus menebak SEMUA 4 label dengan benar untuk dapat 1 poin.
acc_subset = accuracy_score(y_test, y_pred)
print(f"Akurasi (Subset): {acc_subset*100:.2f}%")
print("(Skor ini mengukur seberapa sering model menebak SEMUA 4 label dengan sempurna)")

# Laporan Klasifikasi (Per Label):
# Ini adalah metrik yang JAUH LEBIH BERGUNA.
print("\nLaporan Klasifikasi (Per Label):")
print(classification_report(y_test, y_pred, target_names=label_cols, zero_division=0))

# 7. Simpan Model untuk Aplikasi Prediksi
# Kita perlu menyimpan 3 hal:
# a. Model yang sudah dilatih
# b. Imputer (untuk membersihkan data baru)
# c. Daftar 100 nama fitur (agar tahu fitur apa yang harus diekstrak)

model_filename = 'model_suara.joblib'
imputer_filename = 'imputer.joblib'
features_filename = 'selected_features.joblib'

joblib.dump(model, model_filename)
joblib.dump(imputer, imputer_filename)
joblib.dump(selected_feature_names, features_filename)

print(f"\nModel berhasil disimpan ke: {model_filename}")
print(f"Imputer berhasil disimpan ke: {imputer_filename}")
print(f"Daftar fitur berhasil disimpan ke: {features_filename}")
print("\nAnda sekarang siap membuat aplikasi prediksi!")

## Eksperimen PCA

In [ ]:


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import numpy as np

print("Memulai eksperimen PCA...")

# 1. Pisahkan fitur dan target (menggunakan variabel dari cell kita sebelumnya)
if 'features_df' not in locals():
    print("ERROR: 'features_df' tidak ditemukan. Jalankan Cell 5.")
    raise NameError("'features_df' not defined")

if 'ig_df' not in locals():
    print("ERROR: 'ig_df' tidak ditemukan. Jalankan Cell 6.")
    raise NameError("'ig_df' not defined")
    
# y tidak benar-benar dipakai di cell ini, tapi kita definisikan saja
y = features_df[['label_buka', 'label_tutup', 'label_fikri', 'label_fauzan']]

# === SEMUA FITUR ===
# Ambil semua fitur numerik dari features_df
X_all = features_df.drop(columns=['filename', 'label_buka', 'label_tutup', 'label_fikri', 'label_fauzan'], errors='ignore')
# Pastikan hanya kolom numerik
X_all = X_all.select_dtypes(include=[np.number]).copy()
print(f"Jumlah data (baris): {len(X_all)}")
print(f"Jumlah fitur total: {X_all.shape[1]}")

# === FITUR TERPILIH (TOP 10) ===
# Ambil 10 fitur terbaik dari hasil Information Gain (Cell 6)
# Kode teman Anda menggunakan 10 (iloc[:9] di screenshot Anda salah, harusnya iloc[:10])
top_features = ig_df.head(10)['Feature'].tolist()
X_selected = X_all[top_features]
print(f"Jumlah fitur terpilih (Top 10): {len(top_features)}")


# === 2. IMPUTASI NILAI HILANG (NaN) ===
# Teman Anda pakai 'mean', kita ikuti (meskipun di Cell 6 kita pakai 'median')
print("Melakukan imputasi (mean)...")
X_all = X_all.replace([np.inf, -np.inf], np.nan) # Penting!
X_selected = X_selected.replace([np.inf, -np.inf], np.nan) # Penting!

imputer_all = SimpleImputer(strategy='mean')
imputer_selected = SimpleImputer(strategy='mean')

X_all_imputed = pd.DataFrame(imputer_all.fit_transform(X_all), columns=X_all.columns)
X_selected_imputed = pd.DataFrame(imputer_selected.fit_transform(X_selected), columns=X_selected.columns)

# === 3. NORMALISASI (STANDARD SCALER) ===
# PCA sangat sensitif terhadap skala, jadi ini langkah wajib untuk PCA
print("Melakukan normalisasi (StandardScaler)...")
scaler_all = StandardScaler()
scaler_selected = StandardScaler()

X_all_scaled = scaler_all.fit_transform(X_all_imputed)
X_selected_scaled = scaler_selected.fit_transform(X_selected_imputed)

# === 4. PCA UNTUK SEMUA FITUR ===
print("Melakukan PCA pada semua fitur...")
pca_all = PCA(n_components=None) # Ambil semua komponen
X_all_pca = pca_all.fit_transform(X_all_scaled)

explained_variance_all = pca_all.explained_variance_ratio_
cum_var_all = np.cumsum(explained_variance_all)

# === 5. PCA UNTUK FITUR TERPILIH (TOP 10) ===
print("Melakukan PCA pada fitur terpilih...")
pca_selected = PCA(n_components=None) # Ambil semua komponen
X_selected_pca = pca_selected.fit_transform(X_selected_scaled)

explained_variance_selected = pca_selected.explained_variance_ratio_
cum_var_selected = np.cumsum(explained_variance_selected)

print("\nEksperimen PCA Selesai.")
print(f"Variabel 'cum_var_all' dan 'cum_var_selected' siap untuk visualisasi.")

## Visualisasi PCA

In [ ]:

import matplotlib.pyplot as plt

print("Membuat visualisasi perbandingan PCA...")

# === 1. VISUALISASI PERBANDINGAN PCA ===
plt.figure(figsize=(12, 6))

# Variabel 'cum_var_all' dan 'cum_var_selected' dibuat oleh Cell 8
plt.plot(cum_var_all, label='Semua Fitur', marker='o')
plt.plot(cum_var_selected, label='Fitur Terpilih (Top 10)', marker='s')

plt.title('Perbandingan Cumulative Explained Variance PCA')
plt.xlabel('Jumlah Komponen')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.legend()
plt.grid(True)
plt.show()

# === 2. RINGKASAN HASIL ===
# Variabel-variabel ini juga dibuat oleh Cell 8
print("\nRingkasan PCA:")
print(f"Total komponen (semua fitur): {X_all_pca.shape[1]}")
print(f"Total komponen (fitur terpilih): {X_selected_pca.shape[1]}")
print(f"Explained variance (semua fitur, 5 pertama): {explained_variance_all[:5]}")
print(f"Explained variance (fitur terpilih, 5 pertama): {explained_variance_selected[:5]}")

## Visualisasi PCA 2D & 3D

In [ ]:

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from mpl_toolkits.mplot3d import Axes3D # Untuk plot 3D

print("Memulai visualisasi PCA 2D dan 3D...")

# Pastikan 'X_all_imputed' dan 'y' dari Cell 6 ada
if 'X_all_imputed' not in locals(): # Jika Anda mengikuti alur saya, ini akan jadi X_imputed dari Cell 6
    print("WARNING: 'X_all_imputed' tidak ditemukan. Menggunakan 'X_imputed' dari Cell 6.")
    # Kita pakai X_imputed yang sudah bersih dari Cell 6
    if 'X_imputed' not in locals():
        print("ERROR: 'X_imputed' juga tidak ditemukan. Jalankan Cell 6 terlebih dahulu.")
        raise NameError("'X_imputed' not defined")
    X_for_pca = X_imputed
else: # Jika Anda menggunakan X_all_imputed dari Cell 8
    X_for_pca = X_all_imputed

if 'y_labels' not in locals(): # Jika Anda mengikuti alur saya, ini akan jadi y_labels dari Cell 6
    print("WARNING: 'y_labels' tidak ditemukan. Menggunakan 'y' dari Cell 6.")
    if 'y' not in locals():
        print("ERROR: 'y' juga tidak ditemukan. Jalankan Cell 6 terlebih dahulu.")
        raise NameError("'y' not defined")
    y_for_pca = y
else: # Jika Anda menggunakan y dari Cell 8
    y_for_pca = y_labels

# Pastikan data telah diskalakan untuk PCA
# Kita akan gunakan scaler baru untuk konsistensi di sini
scaler_pca_viz = StandardScaler()
X_scaled_for_viz = scaler_pca_viz.fit_transform(X_for_pca)


# === 1. PCA 2D ===
pca_2d = PCA(n_components=2)
components_2d = pca_2d.fit_transform(X_scaled_for_viz)

df_2d = pd.DataFrame(data=components_2d, columns=['PC1', 'PC2'])
df_2d['label_buka'] = y_for_pca['label_buka'].values
df_2d['label_tutup'] = y_for_pca['label_tutup'].values
df_2d['label_fikri'] = y_for_pca['label_fikri'].values
df_2d['label_fauzan'] = y_for_pca['label_fauzan'].values

# Visualisasi 2D berdasarkan Speaker (Fikri vs Fauzan)
plt.figure(figsize=(10, 7))
sns.scatterplot(
    x="PC1", y="PC2",
    hue=df_2d['label_fikri'].map({0: 'Fauzan', 1: 'Fikri'}), # Map 0/1 ke nama speaker
    style=df_2d['label_fikri'].map({0: 'Fauzan', 1: 'Fikri'}),
    palette='deep',
    data=df_2d,
    legend="full",
    alpha=0.8
)
plt.title('PCA 2D: Speaker (Fikri vs Fauzan)')
plt.xlabel(f'Principal Component 1 ({pca_2d.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca_2d.explained_variance_ratio_[1]*100:.2f}%)')
plt.grid(True)
plt.show()

# Visualisasi 2D berdasarkan Aksi (Buka vs Tutup)
plt.figure(figsize=(10, 7))
sns.scatterplot(
    x="PC1", y="PC2",
    hue=df_2d['label_buka'].map({0: 'Tutup', 1: 'Buka'}), # Map 0/1 ke nama aksi
    style=df_2d['label_buka'].map({0: 'Tutup', 1: 'Buka'}),
    palette='deep',
    data=df_2d,
    legend="full",
    alpha=0.8
)
plt.title('PCA 2D: Aksi (Buka vs Tutup)')
plt.xlabel(f'Principal Component 1 ({pca_2d.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca_2d.explained_variance_ratio_[1]*100:.2f}%)')
plt.grid(True)
plt.show()


# === 2. PCA 3D ===
pca_3d = PCA(n_components=3)
components_3d = pca_3d.fit_transform(X_scaled_for_viz)

df_3d = pd.DataFrame(data=components_3d, columns=['PC1', 'PC2', 'PC3'])
df_3d['label_buka'] = y_for_pca['label_buka'].values
df_3d['label_tutup'] = y_for_pca['label_tutup'].values
df_3d['label_fikri'] = y_for_pca['label_fikri'].values
df_3d['label_fauzan'] = y_for_pca['label_fauzan'].values

# Visualisasi 3D berdasarkan Speaker (Fikri vs Fauzan)
fig_3d_speaker = plt.figure(figsize=(12, 10))
ax_3d_speaker = fig_3d_speaker.add_subplot(111, projection='3d')

# Warna dan marker berdasarkan speaker
colors_speaker = df_3d['label_fikri'].map({0: 'red', 1: 'blue'})
markers_speaker = df_3d['label_fikri'].map({0: 'o', 1: '^'})

for i, txt in enumerate(df_3d.index):
    ax_3d_speaker.scatter(
        df_3d['PC1'][i], df_3d['PC2'][i], df_3d['PC3'][i],
        color=colors_speaker[i],
        marker=markers_speaker[i],
        s=50, alpha=0.7
    )

ax_3d_speaker.set_xlabel(f'Principal Component 1 ({pca_3d.explained_variance_ratio_[0]*100:.2f}%)')
ax_3d_speaker.set_ylabel(f'Principal Component 2 ({pca_3d.explained_variance_ratio_[1]*100:.2f}%)')
ax_3d_speaker.set_zlabel(f'Principal Component 3 ({pca_3d.explained_variance_ratio_[2]*100:.2f}%)')
ax_3d_speaker.set_title('PCA 3D: Speaker (Fikri = biru segitiga, Fauzan = merah bulat)')
plt.show()

# Visualisasi 3D berdasarkan Aksi (Buka vs Tutup)
fig_3d_aksi = plt.figure(figsize=(12, 10))
ax_3d_aksi = fig_3d_aksi.add_subplot(111, projection='3d')

# Warna dan marker berdasarkan aksi
colors_aksi = df_3d['label_buka'].map({0: 'green', 1: 'purple'})
markers_aksi = df_3d['label_buka'].map({0: 's', 1: 'D'}) # s=persegi, D=diamond

for i, txt in enumerate(df_3d.index):
    ax_3d_aksi.scatter(
        df_3d['PC1'][i], df_3d['PC2'][i], df_3d['PC3'][i],
        color=colors_aksi[i],
        marker=markers_aksi[i],
        s=50, alpha=0.7
    )

ax_3d_aksi.set_xlabel(f'Principal Component 1 ({pca_3d.explained_variance_ratio_[0]*100:.2f}%)')
ax_3d_aksi.set_ylabel(f'Principal Component 2 ({pca_3d.explained_variance_ratio_[1]*100:.2f}%)')
ax_3d_aksi.set_zlabel(f'Principal Component 3 ({pca_3d.explained_variance_ratio_[2]*100:.2f}%)')
ax_3d_aksi.set_title('PCA 3D: Aksi (Buka = ungu diamond, Tutup = hijau persegi)')
plt.show()

print("\nVisualisasi PCA 2D dan 3D selesai.")

## Persiapan Dataset Untuk Modeling

In [ ]:

import pandas as pd

print("Mempersiapkan 3 dataset final untuk evaluasi...")

# Ambil variabel-variabel yang sudah kita buat di cell-cell sebelumnya
# Pastikan 'X_selected_imputed', 'X_all_pca', 'X_selected_pca', dan 'y_labels' ada.
if 'y_labels' not in locals():
    print("ERROR: 'y_labels' tidak ditemukan. Jalankan ulang Cell 6.")
    raise NameError("'y_labels' not defined")

# === 1. Dataset hasil Information Gain (fitur terpilih Top 10) ===
# 'X_selected_imputed' dibuat di Cell 8
# 'y_labels' (DataFrame) dibuat di Cell 6
df_ig_ready = pd.concat([
    pd.DataFrame(X_selected_imputed, columns=top_features), 
    y_labels.reset_index(drop=True) # Gabungkan 4 kolom label
], axis=1)


# === 2. Dataset hasil PCA dari semua fitur ===
# 'X_all_pca' dibuat di Cell 8
# Beri nama kolom PC1, PC2, ...
pca_all_cols = [f"PC{i+1}" for i in range(X_all_pca.shape[1])]
df_pca_all_ready = pd.concat([
    pd.DataFrame(X_all_pca, columns=pca_all_cols),
    y_labels.reset_index(drop=True) # Gabungkan 4 kolom label
], axis=1)


# === 3. Dataset hasil PCA dari fitur terpilih (Top 10) ===
# 'X_selected_pca' dibuat di Cell 8
pca_selected_cols = [f"PC{i+1}" for i in range(X_selected_pca.shape[1])]
df_pca_selected_ready = pd.concat([
    pd.DataFrame(X_selected_pca, columns=pca_selected_cols),
    y_labels.reset_index(drop=True) # Gabungkan 4 kolom label
], axis=1)


# === 4. Cetak ringkasan untuk verifikasi ===
print("\nDataset siap pakai:")
print(f"- df_ig_ready: {df_ig_ready.shape} (fitur terpilih IG)")
print(f"- df_pca_all_ready: {df_pca_all_ready.shape} (PCA semua fitur)")
print(f"- df_pca_selected_ready: {df_pca_selected_ready.shape} (PCA fitur terpilih)")

# Verifikasi salah satu dataset
print("\nContoh 5 baris terakhir dari 'df_pca_selected_ready' (termasuk label):")
display(df_pca_selected_ready.tail())

## Modeling dan Hasil Evaluasi

In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === 1. Membuat Kolom Baru untuk Target (Single-Label) ===
print("Membuat kolom 'target' gabungan (single-label)...")

# Kita butuh fungsi untuk mengubah 4 label [1,0,1,0] menjadi string "buka_fikri"
def create_target_string(row):
    aksi = "buka" if row['label_buka'] == 1 else "tutup"
    identitas = "fikri" if row['label_fikri'] == 1 else "fauzan"
    return f"{aksi}_{identitas}"

# Ambil 'df_ig_ready' (atau DataFrame lain, isinya sama 400 baris)
# Terapkan fungsi untuk membuat 1 seri target gabungan
target_series = df_ig_ready.apply(create_target_string, axis=1)

# Tambahkan seri 'target' ini ke 3 DataFrame yang sudah kita siapkan
df_ig_ready['target'] = target_series
df_pca_all_ready['target'] = target_series
df_pca_selected_ready['target'] = target_series

print("Kolom 'target' baru (misal: 'buka_fikri') telah ditambahkan ke 3 DataFrame.")
print(f"Contoh target: {df_ig_ready['target'].unique()}")


# === 2. Membandingkan Models KNN, RF, SVM ===
print("\nMemulai perbandingan model...")

# 1. Pilih dataset yang ingin digunakan (sesuai screenshot teman Anda)
dataset = df_pca_selected_ready
# Anda bisa ganti di atas menjadi:
# dataset = df_ig_ready
# dataset = df_pca_all_ready

print(f"Menggunakan dataset: 'df_pca_selected_ready' (shape: {dataset.shape})")

# 2. Pisahkan fitur dan target
# Kita buang semua kolom label, sisakan fitur (PC1, PC2, ...)
label_cols_to_drop = ['label_buka', 'label_tutup', 'label_fikri', 'label_fauzan', 'target']
X = dataset.drop(columns=label_cols_to_drop, errors='ignore')
y_string = dataset["target"]

# 3. Encode label (misal: 'buka_fikri' -> 0, 'tutup_fikri' -> 1, dst.)
le = LabelEncoder()
y = le.fit_transform(y_string)
print(f"Label telah di-encode. {len(le.classes_)} kelas: {le.classes_}")

# 4. Inisialisasi model-model yang akan dibandingkan
models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM (RBF)": SVC(kernel="rbf", C=1, gamma="scale", random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# 5. Konfigurasi K-fold Cross Validation
# (n_splits=5 berarti 5-fold CV)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6. Evaluasi setiap model
results = []
scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
print("Menjalankan 5-fold Cross Validation untuk setiap model...")

for name, model in models.items():
    scores = cross_validate(
        model, X, y, cv=kfold,
        scoring=scoring_metrics
    )
    
    results.append({
        "Model": name,
        "Akurasi (mean)": np.mean(scores['test_accuracy']),
        "Precision (mean)": np.mean(scores['test_precision_macro']),
        "Recall (mean)": np.mean(scores['test_recall_macro']),
        "F1-Score (mean)": np.mean(scores['test_f1_macro']),
        "std Akurasi": np.std(scores['test_accuracy'])
    })
    
    print(f"  {name:15s} | Acc: {np.mean(scores['test_accuracy']):.4f} | F1: {np.mean(scores['test_f1_macro']):.4f}")

# 7. Simpan hasil ke DataFrame dan tampilkan
print("\n--- Hasil Perbandingan Model ---")
results_df = pd.DataFrame(results).sort_values(by="Akurasi (mean)", ascending=False)
display(results_df)

# 8. Visualisasi hasil perbandingan
print("\nMembuat visualisasi perbandingan akurasi...")
plt.figure(figsize=(8, 5))
plt.bar(
    results_df["Model"], 
    results_df["Akurasi (mean)"], 
    yerr=results_df["std Akurasi"], # Tambahkan error bar
    capsize=5, 
    color=['blue', 'green', 'red']
)
plt.title("Perbandingan Akurasi Model (5-Fold Cross-Validation)")
plt.ylabel("Akurasi (mean)")
plt.xlabel("Model")
plt.ylim(0, 1) # Skala akurasi dari 0 sampai 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Visualisasi Hasil Akhir

In [ ]:
# --- CELL 12: VISUALISASI AKHIR & PENYIMPANAN OBJEK (Pengganti Cell 10 Teman Anda) ---

import pandas as pd
import matplotlib.pyplot as plt
import joblib

# Pastikan 'results_df' ada dari Cell 11
if 'results_df' not in locals():
    print("ERROR: 'results_df' tidak ditemukan. Jalankan ulang Cell 11.")
    raise NameError("'results_df' not defined")

# === 1. VISUALISASI PERBANDINGAN MULTI-METRIK ===
print("Membuat visualisasi perbandingan multi-metrik...")

# Daftar metrik yang ingin kita plot (sesuai dengan kolom di results_df)
metrics = ["Akurasi (mean)", "Precision (mean)", "Recall (mean)", "F1-Score (mean)"]

# Kita "unpivot" DataFrame agar mudah di-plot
results_df_melted = results_df.melt(
    id_vars=["Model"], 
    value_vars=metrics, 
    var_name="Metrik", 
    value_name="Nilai"
)

plt.figure(figsize=(9, 5))
# Loop untuk setiap model dan buat plot garisnya
for model_name in results_df["Model"].unique():
    # Ambil data hanya untuk model ini
    model_data = results_df_melted[results_df_melted["Model"] == model_name]
    
    plt.plot(
        model_data["Metrik"], 
        model_data["Nilai"], 
        marker='o', 
        label=model_name
    )

plt.title("Perbandingan Multi-Metrik Tiap Model (pada Target Single-Label)")
plt.ylabel("Nilai Skor")
plt.ylim(0, 1) # Skala dari 0 sampai 1
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# === 2. MENYIMPAN OBJEK DARI ALUR PCA (Eksperimen Dosen) ===
print("\nMenyimpan objek dari alur eksperimen PCA...")

# Pastikan objek-objek dari Cell 8 ada
try:
    # Buat folder untuk menyimpan model eksperimen
    !mkdir saved_models_eksperimen
    
    # Menyimpan objek PCA dari fitur terpilih
    joblib.dump(pca_selected, "saved_models_eksperimen/pca_selected.joblib")
    
    # Menyimpan objek Scaler dari fitur terpilih
    joblib.dump(scaler_selected, "saved_models_eksperimen/scaler_selected.joblib")
    
    # Menyimpan salah satu model (misal RF) dari Cell 11
    joblib.dump(models["Random Forest"], "saved_models_eksperimen/model_rf_single_label_pca.joblib")
    
    print("Objek PCA, Scaler, dan Model (RF) dari alur eksperimen berhasil disimpan.")

except NameError as e:
    print(f"ERROR: Gagal menyimpan objek. Variabel tidak ditemukan: {e}")
    print("Pastikan Cell 8 dan Cell 11 sudah dijalankan.")
except Exception as e:
    print(f"Terjadi error saat menyimpan: {e}")